In [ ]:
import os
from sklearn.neighbors import KNeighborsRegressor
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import shapefile   
import geopandas as gpd
import sys
import matplotlib.pyplot as plt
import folium
from folium.plugins import MarkerCluster
import warnings
warnings.filterwarnings('ignore')

import sys
# sys.path is a list of absolute path strings
sys.path.append('./flood_tool')

from flood_tool import geo
from flood_tool import tool
from flood_tool import data_preprocessing as dp
from flood_tool import mapping as mp

# # from flood_tool import geo
# # import flood_tool.smote as smote
# from flood_tool import smote
# # from smote import *
# # from . import smote
# # import tool
## READ the unlabelled postcode input

df1 = pd.read_csv('./flood_tool/resources/postcodes_unlabelled.csv')
postcodes = df1.postcode

#sectors =  gpd.read_file( './Distribution/Sectors.shp', low_memory=False)
tool = tool.Tool()


sectors = gpd.read_file("Sectors.shp")

## Visualising Predictions

In [ ]:
risk_prediction = tool.get_flood_class(postcodes, method = 2)
# print(risk_prediction)

df = df1.merge(sectors, how = 'left', left_on = 'sector', right_on = 'name')

df_pred = df.merge(risk_prediction, how = 'left', left_on = 'postcode', right_on = 'postcode')

df_pred = df_pred.dropna()
df_pred = df_pred.drop(['easting_y', 'northing_y'], axis = 1)
#print(df_pred.riskLabel)
gdf = gpd.GeoDataFrame(df_pred)
# gdf.crs
gdf.flood_risk.unique()

In [ ]:
fig  = plt.figure(figsize=(20, 16))



ax2 = plt.subplot()
sectors.plot(ax=ax2, edgecolor= '#000000', facecolor='None', linewidth = 0.1, zorder=3)
gdf.plot(column='flood_risk', k=10, legend=True, ax=ax2, edgecolor='grey', linewidth=0.4, cmap = 'hot') 
ax2.axis('off')
ax2.set_title('Flooding Risk Level prediction per Postcode Sector', fontdict={'fontsize': '20', 'fontweight' : '3'})

In [ ]:
median_price_prediction = tool.get_median_house_price_estimate(postcodes, method = 1)


df = df1.merge(sectors, how = 'left', left_on = 'sector', right_on = 'name')

df_pred_price = df.merge(median_price_prediction, how = 'left', left_on = 'postcode', right_on = 'Postcode')

df_pred_price = df_pred_price.dropna()
#df_pred_price = df_pred.drop(['easting_y', 'northing_y'], axis = 1)

gdf_price = gpd.GeoDataFrame(df_pred_price)


In [ ]:
fig  = plt.figure(figsize=(20, 16))



ax2 = plt.subplot()
sectors.plot(ax=ax2, edgecolor= '#000000', facecolor='None', linewidth = 0.1, zorder=3)
gdf_price.plot(column='Median House Price Prediction', k=10, legend=True, ax=ax2, edgecolor='grey', linewidth=0.4, cmap = 'Reds') 
ax2.axis('off')
ax2.set_title('Median house price predictions', fontdict={'fontsize': '20', 'fontweight' : '3'})

In [ ]:
postcodes_of_interest = postcodes
subset = gdf[gdf.postcode.isin(postcodes_of_interest)]
subset.crs

pred = df_pred_price.merge(df_pred, how = 'left', left_on= 'postcode', right_on= 'postcode' )


pred = gpd.GeoDataFrame(pred)
pred['geometry'] = pred['geometry_x']
pred = pred.drop(['easting_x', 'northing_x','localAuthority_y', 'geometry_y', 'name_y', 'sector_y', 'geometry_x' ], axis = 1)

#Jupyter crashes (webpage re-loads due to large memory usage)
# when trying to plot all the data with Folium
# Hence sample was used to show visualisation
pred['Median House Price Prediction'] = pred['Median House Price Prediction'].round(2)

pred = pred.sample( frac= 0.4)
pred.head()

In [ ]:
annual_flood_risk = tool.get_annual_flood_risk(list(pred.postcode))
annual_flood_risk.head()

In [ ]:
pred = pred.merge(annual_flood_risk, how='inner', left_on = 'postcode', right_on = 'postcodes').dropna()
pred['total value'] = pred['total value'].round(2)
pred['total_risk'] = pred['total_risk'].round(2)
pred.head()

In [ ]:
# Getting locations of pop-ups
locations = tool.get_lat_long(pred.postcode)
# locations

plot_df = gpd.GeoDataFrame(locations.merge(pred, how = 'inner', on = 'postcode'))


In [ ]:
#Creating a pop_up table

def fancy_html(row):
    i = row
    
    Postcode = plot_df['postcode'].iloc[i]                             
    Local_Authority = plot_df['localAuthority_x'].iloc[i]                                                                     
    Flood_Risk = plot_df['flood_risk'].iloc[i]                               
    Median_House_Price = plot_df['Median House Price Prediction'].iloc[i]  
    Total_Risk = plot_df['total_risk'].iloc[i] 
    Total_Value = plot_df['total value'].iloc[i] 

    left_col_colour = "#2A799C"
    right_col_colour = "#C5DCE7"
    
    html = """<!DOCTYPE html>
<html>

    <table style="height: 126px; width: 300px;">
<tbody>
<tr>
<td style="background-color: """+ left_col_colour +""";"><span style="color: #ffffff;">Postcode</span></td>
<td style="width: 200px;background-color: """+ right_col_colour +""";">{}</td>""".format(Postcode) + """
</tr>
<tr>
<td style="background-color: """+ left_col_colour +""";"><span style="color: #ffffff;">Local_Authority</span></td>
<td style="width: 200px;background-color: """+ right_col_colour +""";">{}</td>""".format(Local_Authority) + """
</tr>
<tr>
<td style="background-color: """+ left_col_colour +""";"><span style="color: #ffffff;">Flood_Risk</span></td>
<td style="width: 200px;background-color: """+ right_col_colour +""";">{}</td>""".format(Flood_Risk) + """
</tr>
<tr>
<td style="background-color: """+ left_col_colour +""";"><span style="color: #ffffff;">Median_House_Price</span></td>
<td style="width: 200px;background-color: """+ right_col_colour +""";">{}</td>""".format(Median_House_Price) + """
</tr>
<tr>
<td style="background-color: """+ left_col_colour +""";"><span style="color: #ffffff;">Total_Risk</span></td>
<td style="width: 200px;background-color: """+ right_col_colour +""";">{}</td>""".format(Total_Risk) + """
</tr>
<tr>
<td style="background-color: """+ left_col_colour +""";"><span style="color: #ffffff;">Total_Value</span></td>
<td style="width: 200px;background-color: """+ right_col_colour +""";">{}</td>""".format(Total_Value) + """

</tbody>
</table>
</html>
"""
    return html

In [ ]:
import branca
color = 'red'

m = folium.Map()
xmin, ymin, xmax, ymax = plot_df.total_bounds
m.fit_bounds([[ymin, xmin], [ymax, xmax]])

for i in range(0,len(plot_df)):
    html = fancy_html(i)
 
    iframe = branca.element.IFrame(html=html,width=300,height=200)
    popup = folium.Popup(iframe,parse_html=True)
    
    folium.Marker([plot_df['Latitude'].iloc[i],plot_df['Longitude'].iloc[i]],
                  popup=popup,icon=folium.Icon(color=color, icon='info-sign')).add_to(m)
for _, r in plot_df.iterrows():
    # Without simplifying the representation of each borough,
    # the map might not be displayed
    sim_geo = gpd.GeoSeries(plot_df['geometry']).simplify(tolerance=0.001)
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data=geo_j,
                           style_function=lambda x: {'fillColor': 'None'})

    geo_j.add_to(m)

m

In [ ]:
df_pred_price
East_data=df_pred_price.easting
North_data=df_pred_price.northing
loc=[]
for E, N in zip(East_data, North_data):
    lat = geo.get_gps_lat_long_from_easting_northing(E, N, rads=False, dms=False)[0][0] 
    long = geo.get_gps_lat_long_from_easting_northing(E, N, rads=False, dms=False)[1][0]
    loc.append((lat, long))

import numpy as np


locations = loc


In [ ]:
plot_df.shape

In [ ]:
import matplotlib.pyplot as plt

import folium
from folium.plugins import MarkerCluster
m1=folium.Map(location=[54, -0.5],zoom_start=11,min_zoom=6,max_zoom=14)
folium.GeoJson(sectors).add_to(m1)
folium.TileLayer('Stamen Toner').add_to(m1)
marker_cluster = MarkerCluster(locations)
marker_cluster.add_to(m1)
folium.LayerControl().add_to(m1)
m1

## Rainfall Visualisation

In [ ]:
typ_day = pd.read_csv('./flood_tool/resources/typical_day.csv')

In [ ]:
rainfall= dp.get_rainfall_data_processed(typ_day)
rainfall.sort_values(['value'],ascending=False)

In [ ]:
# Import the station data
stations = pd.read_csv('./flood_tool/resources/stations.csv')
stations.dropna(inplace=True)
stations.head()

## Rainfall hourly for a typical day

In [ ]:
rainfall_h = dp.get_rainfall_per_hour_data(rainfall, stations)
highrain_stations = rainfall_h.loc[rainfall_h.class_rain != 'slight', :]
lat, lon, classe = highrain_stations.latitude.unique(), highrain_stations.longitude.unique(), highrain_stations.class_rain.unique()

## Rainfall hourly for a wet day

In [ ]:
# Import the typical_day data & pre-process
wet_day = pd.read_csv('./flood_tool/resources/wet_day.csv')
wet_day = dp.get_rainfall_data_processed(wet_day)
wet_day_h = dp.get_rainfall_per_hour_data(wet_day, stations)
heavy_rain_wetd = wet_day_h.loc[wet_day_h.class_rain != 'slight', :]
heavy_rain_wetd.drop(['dateTime', 'value_y', 'min'], axis=1, inplace=True)
heavy_rain_wetd.drop_duplicates(inplace=True)


### Plotting the station where the rainfall per hour pour a typical day is moderate, heavy or violent

In [ ]:
def generate_popup(elem):
    html = f'''<b1>Typicial day rainfall : <br />\
    <b>{elem}</b>\
    '''

    iframe = folium.IFrame(html,
                           width=400,
                           height=50)

    popup = folium.Popup(iframe,
                         max_width=150)
    
    return popup

In [ ]:
map = mp.plot_circle(lat[0], lon[0], radius=4000, zoom_start = 6.5)
folium.Marker(location=(lat[0], lon[0]), 
                  popup=generate_popup(classe[0]),
                  icon=folium.Icon(color='lightblue', icon='info-sign')).add_to(map)

folium.Circle(location=[lat[1], lon[1]], radius=4000, fill=True,
                  fillOpacity=0.6).add_to(map)
folium.Marker(location=(lat[1], lon[1]), 
                  popup=generate_popup(classe[0]),
                  icon=folium.Icon(color='lightblue', icon='info-sign')).add_to(map)

folium.Circle(location=[lat[2], lon[2]], radius=4000, fill=True,
                  fillOpacity=0.6, color='darkblue').add_to(map)
folium.Marker(location=(lat[2], lon[2]), 
                  popup=generate_popup(classe[1]),
                  icon=folium.Icon(color='darkblue', icon='info-sign')).add_to(map)

map

### Plotting the station where the rainfall per hour for a wet day is moderate, heavy or violent

In [ ]:
import folium

map = mp.plot_circle(heavy_rain_wetd.iloc[0,9], heavy_rain_wetd.iloc[0,10], radius=4000, zoom_start = 6.5)
folium.Marker(location=(heavy_rain_wetd.iloc[0,9], heavy_rain_wetd.iloc[0,10]), 
                  popup=generate_popup(heavy_rain_wetd.iloc[0,3]),
                  icon=folium.Icon(color='lightblue', icon='info-sign')).add_to(map)

for i in range(1,len(heavy_rain_wetd.loc[heavy_rain_wetd.class_rain == 'moderate', :])):
    folium.Circle(location=[heavy_rain_wetd.iloc[i,9], heavy_rain_wetd.iloc[i,10]], radius=4000, fill=True,
                  fillOpacity=0.6, color='lightblue').add_to(map)
    folium.Marker(location=(heavy_rain_wetd.iloc[i,9], heavy_rain_wetd.iloc[i,10]), 
                  popup=generate_popup('moderate'),
                  icon=folium.Icon(color='lightblue', icon='info-sign')).add_to(map)

for i in range(0,len(heavy_rain_wetd.loc[heavy_rain_wetd.class_rain == 'heavy', :])):
    folium.Circle(location=[heavy_rain_wetd.iloc[i,9], heavy_rain_wetd.iloc[i,10]], radius=4000, fill=True,
                  fillOpacity=0.6, color='darkblue').add_to(map)
    folium.Marker(location=(heavy_rain_wetd.iloc[i,9], heavy_rain_wetd.iloc[i,10]), 
                  popup=generate_popup('heavy'),
                  icon=folium.Icon(color='darkblue', icon='info-sign')).add_to(map)

for i in range(0,len(heavy_rain_wetd.loc[heavy_rain_wetd.class_rain == 'violent', :])):
    folium.Circle(location=[heavy_rain_wetd.iloc[i,9], heavy_rain_wetd.iloc[i,10]], radius=4000, fill=True,
                  fillOpacity=0.6, color='darkred').add_to(map)
    folium.Marker(location=(heavy_rain_wetd.iloc[i,9], heavy_rain_wetd.iloc[i,10]), 
                  popup=generate_popup('violent'),
                  icon=folium.Icon(color='darkred', icon='info-sign')).add_to(map)

map

## Map Rivers typical day

In [ ]:
# Import the typical_day data
typ_day = pd.read_csv('./flood_tool/resources/typical_day.csv')


In [ ]:
# Import the station data
stations = pd.read_csv('./flood_tool/resources/stations.csv')
stations.dropna(inplace=True)
stations.head()

## Rivers' height evolution for a typical day

In [ ]:

rivers_typ = dp.get_level_data_processed(typ_day)

In [ ]:
rivers_typ = dp.get_level_data_aggregate(rivers_typ, stations)
rivers_std_typ = rivers_typ.sort_values(('value', 'std'), ascending = False)[0:5]

## Rivers' height evolution for a wet day

In [ ]:
# Import the typical_day data
wet_rivers_day = pd.read_csv('./flood_tool/resources/wet_day.csv')
rivers_wd = dp.get_level_data_processed(wet_rivers_day)
rivers_wd_agg = dp.get_level_data_aggregate(rivers_wd, stations)

In [ ]:
rivers_std = rivers_wd_agg.sort_values(('value', 'std'), ascending = False)[0:5]

### Plotting the top five station where the rivers height can change in a day for a wet day

In [ ]:
def generate_popup(elem):
    html = f'''<b1>Rivers mASD standard dev. : <br />\
    <b>{elem}</b>\
    '''

    iframe = folium.IFrame(html,
                           width=300,
                           height=50)

    popup = folium.Popup(iframe,
                         max_width=200)
    
    return popup

In [ ]:
import folium

map = flood_tool.plot_circle(rivers_std.iloc[0,4], rivers_std.iloc[0,5], radius=4000, zoom_start = 6.5)
folium.Marker(location=(rivers_std.iloc[0,4], rivers_std.iloc[0,5]), 
                  popup=generate_popup(round(rivers_std.iloc[0,2], 2)),
                  icon=folium.Icon(color='darkgreen', icon='info-sign')).add_to(map)

for i in range(1,len(rivers_std)):
    folium.Circle(location=[rivers_std.iloc[i,4], rivers_std.iloc[i,5]], radius=4000, fill=True,
                  fillOpacity=0.6, color='green').add_to(map)
    folium.Marker(location=(rivers_std.iloc[i,4], rivers_std.iloc[i,5]), 
                  popup=generate_popup(round(rivers_std.iloc[i,2], 2)),
                  icon=folium.Icon(color='green', icon='info-sign')).add_to(map)

map

### Plotting the top five station where the rivers height can change in a day for a typical day

In [ ]:
import folium

map = flood_tool.plot_circle(rivers_std_typ.iloc[0,4], rivers_std_typ.iloc[0,5], radius=4000, zoom_start = 6.5)
folium.Marker(location=(rivers_std_typ.iloc[0,4], rivers_std_typ.iloc[0,5]), 
                  popup=generate_popup(round(rivers_std_typ.iloc[0,2], 2)),
                  icon=folium.Icon(color='darkgreen', icon='info-sign')).add_to(map)

for i in range(1,len(rivers_std_typ)):
    folium.Circle(location=[rivers_std_typ.iloc[i,4], rivers_std_typ.iloc[i,5]], radius=4000, fill=True,
                  fillOpacity=0.6, color='green').add_to(map)
    folium.Marker(location=(rivers_std_typ.iloc[i,4], rivers_std_typ.iloc[i,5]), 
                  popup=generate_popup(round(rivers_std_typ.iloc[i,2], 2)),
                  icon=folium.Icon(color='green', icon='info-sign')).add_to(map)

map